In [1]:
from bs4 import BeautifulSoup
from urllib3 import PoolManager
import logging

from multiprocessing import Pool
import os 
path="/home/jurkis/leto2021/slovak/data/testbed"
files = os.listdir(path)
import pandas as pd


In [70]:
with open(os.path.join(path,'kmet-stefan_na-vychode_txt.html.utf'),'r') as f:
    data = f.read()



In [71]:
soup =BeautifulSoup(data)

In [72]:
for chap in soup.find_all("div",class_="chapter"):
    print(len(chap.text.split("\n")))

7
63
54
26
14
28
120
76
102
91
32
133
91
38
52
27
158
45
60
56
113
21
30
59
38
31
27
14
119
120
20
17
100
12
88
319


In [73]:
import re
# pat = re.compile(r"\n\ {2,}")
def parse_k(name):
    book={}
    chapters = []
    try:
        with open(os.path.join(path,name),'r') as f:
            data = f.read()
    except Exception:
        print(f"Problem with encoding {name}")
        return {}
    soup = BeautifulSoup(data)
    
    for chap in soup.find_all('div',class_="chapter"):
        parags=[]
        for p in chap.find_all('p'):
            if p.text.startswith('['):
                continue
            parags.append(p.text)
        chapters.append("\n".join(parags))
        
    cleaned = [re.sub(r"\n\ {2,}",r' ',s) for s in chapters]
    book['chapters'] = cleaned
    book['name'] = name 
    return book

In [76]:
with Pool(8) as p:
    res = p.map(parse_k,files)

Problem with encoding feldek_mahuja_txt.htmlProblem with encoding kompis-peter_ked-sa-mrtvy-vrati_txt.htmlProblem with encoding zilinsky-michal_zivot-a-dielo-daniela-krmana_txt.html

Problem with encoding brocko-jan_chlieb-kazdodenny-partizansky_txt.html
Problem with encoding sur-jan_pytac-z-panskeho-rodu_txt.html

Problem with encoding gasparik-lestinsky_pod-lipkou_txt.html
Problem with encoding zatratene-zlato1_txt.html
Problem with encoding andersen_pohadky3_txt.html
Problem with encoding royova-kristina_otcovrah_txt.htmlProblem with encoding benovsky_txt.html

Problem with encoding matzenauer_vdacne-siroty_txt.html
Problem with encoding mucaji-juraj_na-cestach_txt.html
Problem with encoding rizner_rozpravocky-zo-zivota-slavnych-muzov_txt.html
Problem with encoding kalinciak-jan_put-lasky_txt.html
Problem with encoding vajansky-svetozar-hurban_sucha-ratolest_txt.html
Problem with encoding matzenauer_za-styri-cvoky_txt.html
Problem with encoding capek-josef_umeni_txt.html
Problem wit

In [79]:
import pandas as pd
df = pd.DataFrame.from_records(results)


In [81]:
df['concat'] = df['chapters'].apply(lambda x: "\n".join([y.strip() for y in x]).strip())

In [86]:
df['length'] = df['concat'].map(len)

In [5]:
from spacy.tokenizer import Tokenizer
from spacy.lang.sk import Slovak

nlp = Slovak()
tokenizer = Tokenizer(nlp.vocab)

tokens = tokenizer(df['concat'].iloc[0])

In [ ]:
# df.to_csv("concats.zlaty")
from tqdm import tqdm

all_toks=[]
for doc in tqdm(df['concat']):
    all_toks.append(tokenizer(doc))



 29%|██▊       | 614/2147 [00:11<03:37,  7.05it/s] 

In [3]:
import pandas as pd
df = pd.read_csv("concats.zlaty")

In [6]:
df['toks'] = df['concat'].apply(lambda x: tokenizer(x))

In [8]:
pd.DataFrame(df['toks']).to_csv("tokens.csv")